In [4]:
from huggingface_hub import login

# Introduce tu token personal
hugging_face_token = ""

# Loguearte en Hugging Face
login(token=hugging_face_token)
print("Logged in successfully!")

Logged in successfully!


In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, GenerationConfig

class Llama3:
    def __init__(self, model_path):
        self.model_id = model_path

        # Cargar el tokenizador
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)
        
        # Cargar el modelo
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            torch_dtype=torch.float16,
            device_map="auto",
        )

        # Verificar y asignar tokens especiales
        if self.tokenizer.eos_token_id is None:
            self.tokenizer.eos_token_id = self.tokenizer.convert_tokens_to_ids("</s>")
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
        
        # Crear configuración de generación
        self.generation_config = GenerationConfig.from_pretrained(self.model_id)
        self.generation_config.eos_token_id = self.tokenizer.eos_token_id
        self.generation_config.pad_token_id = self.tokenizer.pad_token_id

        # Crear pipeline
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            config=self.generation_config
        )

    def get_response(self, query, message_history=[], max_tokens=100000, temperature=0.7, top_p=0.9):
        user_prompt = message_history + [{"role": "user", "content": query}]
        prompt = self.tokenizer.apply_chat_template(
            user_prompt, tokenize=False, add_generation_prompt=True
        )
        
        # Generar texto
        outputs = self.pipeline(
            prompt,
            max_new_tokens=max_tokens,
            eos_token_id=self.tokenizer.eos_token_id,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
        )
        
        response = outputs[0]["generated_text"][len(prompt):]
        return response, user_prompt + [{"role": "assistant", "content": response}]

    def chatbot(self, system_instructions=""):
        conversation = [{"role": "system", "content": system_instructions}]
        while True:
            user_input = input("User: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Cerrando el chatbot. Goodbye!")
                break
            response, conversation = self.get_response(user_input, conversation)
            print(f"Assistant: {response}")

    def chatbot_without_memory(self, system_instructions=""):
        conversation = [{"role": "system", "content": system_instructions}]
        while True:
            user_input = input("User: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Cerrando el chatbot. Goodbye!")
                break
            response, _ = self.get_response(user_input, conversation)
            print(f"Assistant: {response}")

    def prompt_without_memory(self, prompt, system_instructions=""):
        conversation = [{"role": "system", "content": system_instructions}]
        response, _ = self.get_response(prompt, conversation)
        return response

In [5]:
# if _name_ == "_main_":
torch.cuda.empty_cache()

bot = Llama3("meta-llama/Meta-Llama-3-8B-Instruct")
# bot.chatbot_without_memory()

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [9]:
INSTRUCTION = """INPUT: 
Eres un modelo que recibiras una lista de textos donde cada linea sera de la forma: indice|texto

PROCESO QUE DEBES REALIZAR:
Solo debes evaluar las siguientes cosas:
 - similarity: Boleano que indica si todos los textos son semanticamente similares (Valores: "True" cuando es similar o "False" si no lo es)
 - cluster_name: La oración, Frase o Palabra que representa de mejor forma al conjunto de datos brindado, debe ser lo mas
                 representativo posible.

RESPUESTA:

Unica linea, sin explicarme nada, solo la respuesta pedida con la forma de similarity|cluster_name
"""

In [ ]:
# result='''398 | falta pavimentacion carreteras
# 3325 | falta pavimentacion mejoramiento carretera
# 3826 | falta pavimentacion calles
# 4048 | falta pavimentacion caminos caserio
# 4734 | falta pavimentacion calles'''

In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv("/kaggle/input/cluster-pipexp-1/df_clusters_exp_004.csv", index_col=0)
n_clusters = len(np.unique(df.Cluster)) - 1

df['PrimaryLabel'] = ""
for cluster in range(n_clusters):
    result = ''
    count = 0
    for index, row in df[df.Cluster == cluster].iterrows():
      result += f"{index} | {row['t1txt']}\n"
      count += 1
      if count > 700:
          break
    
    response = bot.prompt_without_memory(result, INSTRUCTION)
    print(response)
    similarity, cluster_name = response.split('|')

    if similarity:
        df.loc[df.Cluster == cluster, 'PrimaryLabel'] = cluster_name.lower()

True|NO SABE
True|DESASTRES NATURALES
True|LA DESNUTRICION
True|LA DESNUTRICION INFANTIL
True|ALCOHOLISMO
True|INVASION DE VENEZOLANOS
True|NARCOTRAFICO
True|TERRORISMO
True|FEMINICIDIO


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


True|FEMINICIDIOS
True|DROGADICCION
True|ACCIDENTES DE TRANSITO
True|ALCOHOLISMO DROGADICCION
False|PROSTITUCION
True|VIOLACION SEXUAL
True|VIOLENCIA SEXUAL
True|FALTA DE COBERTURA DE SERVICIOS BASICOS
True|MIGRACION VENEZOLANA
True|VIOLACIONES
True|FALTA DE JUSTICIA
True|FALTA DE SERVICIOS BASICOS
True|FALTA DE APOYO AL PERUANO
True|MIGRACION DE VENEZOLANOS
True|MIGRACION DE VENEZOLANOS
True|VIOLACION
False|QUE SE VAYA LA MINA
True|CONTAMINACION AMBIENTAL
True|MINERIA INFORMAL
True|FALTA DE SANEAMIENTO BASICO
True|CONTAMINACION MINERA
True|CONTAMINACION DE LA MINERIA
True|FALTA DE AGUA
True|LA CONTAMINACION AMBIENTAL
True|DESIGUALDAD
True|INJUSTICIA
True|INGRESO DE VENEZOLANOS
True|MIGRACION EXTRANJERA
True|FALTA DE ELECTRIFICACION
True|CONTAMINACION DEL AGUA
True|FALTA DE VALORES
True|VIOLENCIA A LA MUJER
True|MIGRACION DE EXTRANJEROS
True|FALTA DE SERVICIOS BASICOS AGUA DESAGUE
True|FALTA DE SERVICIOS DE AGUA
True|INGRESO DE VENEZOLANOS AL PAIS
True|LOS VENEZOLANOS
True|ANALFABETISM

In [11]:
df[df.Cluster == 19]

,t1txt,clean,Cluster,PrimaryLabel
395,MALA ADMINISTRACION DE LA JUSTICIA,mala administracion justicia,19,falta de justicia
824,FALTA DE JUSTICIA PARA LAS PERSONAS VULNERABLES,falta justicia persona vulnerables,19,falta de justicia
865,FALTA DE JUSTICIA,falta justicia,19,falta de justicia
889,FALTA DE JUSTICIA,falta justicia,19,falta de justicia
897,MALA ADMINISTRACION DE LA JUSTICIA,mala administracion justicia,19,falta de justicia
...,...,...,...,...
7760,FALTA DE JUSTICIA,falta justicia,19,falta de justicia
7831,NO HAY JUSTICIA EN LAS AUTORIDADES,justicia autoridades,19,falta de justicia
7917,MAL FUNCIONAMIENTO DEL SISTEMA DE JUSTICIA,mal funcionamiento sistema justicia,19,falta de justicia
8086,LA FALTA DE JUSTICIA,falta justicia,19,falta de justicia


In [12]:
INSTRUCTION_2 = """INPUT: 
Eres un modelo que recibira una lista de etiquetas, donde cada linea tendra la siguiente forma: identificador|etiqueta

PROCESO QUE DEBES REALIZAR:
Para la lista de etiqueta que te mande, necesito que formes grupos de etiquetas por la cercania o similitud semantica, debido a que puede haber
etiquetas que sean muy similares o semanticamente similares o se refieren a un tópico o tema similar, y a cada uno de estos nuevos grupos formados
debes darle un nombre de grupo que describa de forma conscisa, clara, precisa, bien escrita y que sea representativo de todas las etiquetas que
conforma.

RESTRICCIONES:
- Cada grupo que formes debe ser disjuntas entre ellas, es decir, cada etiqueta solo puede ser parte de uno de los grupos.
- Todas las etiquetas deben ser parte de algun grupo.

RESPUESTA:
Lista de los nuevos grupos que haz formado, donde cada linea tiene la forma: grupo|nombre de grupo|lista de identificadores
"""

In [13]:
df.PrimaryLabel = df.PrimaryLabel.str.lower()

In [ ]:
# categories_dict = {}

# # Procesar línea por línea
# current_category = None
# for line in text_data.strip().splitlines():
#     if line.endswith(':'):  # Detectar categorías
#         current_category = line[:-1]  # Remover el ':' final
#         categories_dict[current_category] = []
#     elif line.startswith('-'):  # Detectar etiquetas
#         label = line[1:].strip()  # Remover el '-' inicial y espacios
#         if current_category:
#             categories_dict[current_category].append(label)

In [14]:
df['SecundaryLabel'] = ""

temp = df[df['PrimaryLabel'] != ''][['Cluster', 'PrimaryLabel']]
temp = temp.drop_duplicates()

labels = ''
for index, row in temp.iterrows():
  labels += f"{row['Cluster']}|{row['PrimaryLabel']}\n"

response = bot.prompt_without_memory(labels, INSTRUCTION_2)
print(response)


Después de analizar las etiquetas, he formado los siguientes grupos y les he asignado un nombre que describa de forma concisa, clara, precisa y representativa de todas las etiquetas que componen cada grupo:

**Grupo 1: Violencia y Delincuencia**
| grupo | nombre de grupo | lista de identificadores
| --- | --- | --- |
| 1 | Violencia y Delincuencia | 68, 8, 24, 40, 14, 69, 51, 9, 65, 19, 18, 107, 49, 15, 50, 15

**Grupo 2: Problemas Sociales y Económicos**
| grupo | nombre de grupo | lista de identificadores
| --- | --- | --- |
| 2 | Problemas Sociales y Económicos | 57, 33, 89, 66, 52, 93, 62, 32, 112, 101, 96, 101, 98, 43, 38, 46, 53, 104, 103

**Grupo 3: Medio Ambiente y Salud**
| grupo | nombre de grupo | lista de identificadores
| --- | --- | --- |
| 3 | Medio Ambiente y Salud | 56, 26, 32, 55, 28, 16, 80, 79, 29, 30, 59, 38, 17

**Grupo 4: Inmigración y Integración**
| grupo | nombre de grupo | lista de identificadores
| --- | --- | --- |
| 4 | Inmigración y Integración | 22, 41, 

In [ ]:
print(response)

In [ ]:
import re
from collections import defaultdict

# Datos de ejemplo (tu texto de entrada)
data = response
# Inicializar estructura para almacenar categorías e IDs
categories = defaultdict(set)

# Procesar línea por línea
current_category = None
for line in data.strip().splitlines():
    if re.match(r"^\*\*\d+\.\s", line):  # Detectar categoría (líneas que comienzan con "**1.")
        current_category = re.sub(r"^\*\*\d+\.\s|\*\*", "", line).strip()  # Limpiar texto de la categoría
    elif line.startswith("*"):  # Detectar etiquetas
        # Extraer IDs
        id_match = re.search(r"\((.+)\)", line)
        if id_match:
            ids = id_match.group(1)
            id_list = [int(i.strip()) for i in ids.split(",")]  # Convertir IDs a lista de enteros
            categories[current_category].update(id_list)  # Agregar IDs al conjunto

# Convertir los valores a listas ordenadas
categories = {key: sorted(list(value)) for key, value in categories.items()}

# Imprimir resultado estructurado
for category, ids in categories.items():
    # print(f"Categoría: {category}")
    # print(f"  IDs: {ids}")
    print(f"{category}")
    print(f"{len(ids)}")


In [ ]:
for category, ids in categories.items():
    # print(f"Categoría: {category}")
    # print(f"  IDs: {ids}")
    print(f"{category}")
    print(f"{len(ids)}")

In [ ]:
for category, ids in categories.items():
    df.loc[df['Cluster'].isin(ids), 'SecundaryLabel'] = category

In [ ]:
df[df.SecundaryLabel != '']

## Prueba con T5

In [ ]:
flax-community/spanish-t5-small

In [ ]:
from transformers import T5Tokenizer, T5Model

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5Model.from_pretrained("t5-small")

In [ ]:
# Definir el prompt y el contexto
prompt = "¿Hola, cómo estás? ¿Entiendes español?"
context = ""

# Preparar la entrada para el modelo
input_text = f"context: {context} prompt: {prompt}"  # Formato más claro y limpio
input_ids = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True).input_ids

# Generar la respuesta
outputs = model.generate(
    input_ids, 
    max_length=50,              # Limitar la longitud de la respuesta
    num_beams=1,                # Más haces para mejorar la calidad
    early_stopping=True,        # Detenerse temprano si es suficiente
    no_repeat_ngram_size=2      # Evitar repeticiones de palabras
)

# Decodificar la respuesta
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Imprimir la respuesta
print("Answer:\t", response)

In [ ]:
from transformers import T5Tokenizer, T5Model

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5Model.from_pretrained("t5-small")